In [1]:
print("OK")

OK


In [2]:
%pwd

'd:\\SIHPrototype\\Aarogya-Mitra\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\SIHPrototype\\Aarogya-Mitra'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\ASUS\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    
    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [10]:
#extracted_data

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 1122


In [14]:
#text_chunks

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
#Download the Embeddings from Hugging Face

def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21172\2825795530.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [33]:
#query_result

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [17]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="pcsk_hmyyY_LgJc3XvMC1hCTSoEW3pRENtkS7mnCeJdaotrfdqtnYHaBnVYbad8KRw7eUbg6h9")

index_name = "healio"


pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "healio",
    "metric": "cosine",
    "host": "healio-r38hg8y.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [18]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

c:\Users\ASUS\anaconda3\envs\medibot\lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [24]:
# Load Existing Index
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [20]:
docsearch

In [21]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [22]:
retrieved_docs = retriever.invoke("What is habit?")

In [23]:
retrieved_docs

[Document(id='86550413-bf65-4c13-a988-fb3a1e07d6b2', metadata={'author': 'James Clear', 'creationdate': '2020-04-30T18:46:22+00:00', 'creator': 'calibre 3.48.0 [https://calibre-ebook.com]', 'page': 10.0, 'page_label': '11', 'producer': 'calibre 3.48.0 [https://calibre-ebook.com]', 'source': 'Data\\Atomic habits ( PDFDrive ).pdf', 'title': 'Atomic habits \\( PDFDrive.com \\).pdf', 'total_pages': 256.0}, page_content='classroom\tas\tI\timproved\tmy\tstudy\thabits\tand\tmanaged\tto\tearn\tstraight\tA’s\nduring\tmy\tfirst\tyear.\nA\thabit\tis\ta\troutine\tor\tbehavior\tthat\tis\tperformed\tregularly—and,\tin\tmany\ncases,\tautomatically.\tAs\teach\tsemester\tpassed,\tI\taccumulated\tsmall\tbut\nconsistent\thabits\tthat\tultimately\tled\tto\tresults\tthat\twere\tunimaginable\tto\tme\nwhen\tI\tstarted.\tFor\texample,\tfor\tthe\tfirst\ttime\tin\tmy\tlife,\tI\tmade\tit\ta\thabit\tto\tlift\nweights\tmultiple\ttimes\tper\tweek,\tand\tin\tthe\tyears\tthat\tfollowed,\tmy\tsix-foot-'),
 Document(id

In [24]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

In [29]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question_answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't have information. Use three sentences maximum and keep the "
    "answer concise"
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [30]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "What is habit?"})
print(response["answer"])

I am sorry, but this document does not contain information about acne.
